In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_naukri_jobs():
    url = "https://www.naukri.com/"
    job_title = "Data Scientist"
    location_filter = "Delhi/NCR"
    salary_filter = "3-6 Lakhs"
    num_jobs_to_scrape = 10
    
    # Step 1: Search for Data Scientist jobs
    search_url = f"{url}data-scientist-jobs"
    response = requests.get(search_url)
    if response.status_code != 200:
        print("Failed to fetch the search results page")
        return
    
    # Step 2: Apply location and salary filters
    soup = BeautifulSoup(response.content, "html.parser")
    location_filter_checkbox = soup.find("input", id="chk-Delhi / NCR-cityType")
    salary_filter_checkbox = soup.find("input", id="chk-3-6 Lakhs-annualSalary")
    
    if location_filter_checkbox:
        location_filter_checkbox['checked'] = 'checked'
    if salary_filter_checkbox:
        salary_filter_checkbox['checked'] = 'checked'
    
    # Step 3: Fetch the filtered search results
    filtered_response = requests.post(search_url, data={'cityType': 'Delhi / NCR', 'annualSalary': '3-6 Lakhs'})
    if filtered_response.status_code != 200:
        print("Failed to fetch the filtered search results page")
        return
    
    # Step 4: Scrape data for the first 10 jobs
    jobs_data = []
    filtered_soup = BeautifulSoup(filtered_response.content, "html.parser")
    jobs = filtered_soup.find_all("article", class_="jobTuple bgWhite br4 mb-8")
    
    for job in jobs[:num_jobs_to_scrape]:
        job_title = job.find("a", class_="title fw500 ellipsis").text.strip()
        company_name = job.find("a", class_="subTitle ellipsis fleft").text.strip()
        job_location = job.find("li", class_="fleft grey-text br2 placeHolderLi location").text.strip()
        experience_required = job.find("li", class_="fleft grey-text br2 placeHolderLi experience").text.strip()
        
        jobs_data.append({
            "Job Title": job_title,
            "Company Name": company_name,
            "Job Location": job_location,
            "Experience Required": experience_required
        })
    
    return jobs_data

if __name__ == "__main__":
    jobs = scrape_naukri_jobs()
    jobs_df = pd.DataFrame(jobs)
    print(jobs_df)
